<img align="center" src="../extra/logos/logos.png" width='800px'>

# Operaciones con el DataCube

>Nota: Parte de este notebook es una adaptación realizada las guías de [DataCube Australia](https://github.com/GeoscienceAustralia/dea-notebooks).

## 1. Combinar productos

## 2. Operaciones con Xarray

## 

# Mini - Tarea

Repetir el mismo 